In [5]:
from GodotExperimentWrapper import GodotExperimentWrapper
import pandas as pd
import json
import math
import random

def clamp_HdgD(hdg):      
    if hdg > 180:
        return -(360 - hdg)        
    
    if hdg < -180:
        return (360 + hdg )    
    return hdg

def generate_cases(blue_altitude, red_altitude, angle_off, aspect_angle, min_dist=5, max_dist=50, dist_step=0.5):
    
    cases = []
    dist_cases = [x / 10 for x in range(min_dist * 10, max_dist * 10, int(dist_step * 10))]    
       
    for distance in dist_cases:
        
        blue_hdg = clamp_HdgD(aspect_angle)
        red_hdg = clamp_HdgD(blue_hdg + angle_off)                 
        
        case = {
            'agents_config': {
                'blue_agents': {
                    'init_position': {'x': 0.0, 'y': blue_altitude, 'z': 30.0},                    
                    'init_hdg': blue_hdg,                    
                },    
                'red_agents': {
                    'init_position': {'x': 0.0, 'y': red_altitude, 'z': 30.0 - distance},                    
                    'init_hdg': red_hdg,                    
                }                
            }
        }
        cases.append(case)

    return cases


def generate_random_cases(blue_altitude_range, red_altitude_range, angle_off_range, aspect_angle_range):
    blue_alt = random.uniform(blue_altitude_range[0], blue_altitude_range[1])
    red_alt = random.uniform(red_altitude_range[0], red_altitude_range[1])
    angle_off = random.uniform(angle_off_range[0], angle_off_range[1])
    aspect_angle = random.uniform(aspect_angle_range[0], aspect_angle_range[1])

    condition = {
        "blue_alt": blue_alt,
        "red_alt": red_alt,
        "angle_off": angle_off,
        "aspect_angle": aspect_angle
    }

    return condition

config_dict =   { 'EnvConfig' : {
                            'task': 'b_ace_v1',
                            'env_path': 'BVR_AirCombat/bin/B_ACE_v4.exe',
                            'renderize': 1,
                            'experiment_mode'  : 1,
                            'parallel_envs': 1,
                            'max_cycles': 1800,
                            'seed': 1,
                            'port': 12500,
                            'action_repeat': 20,
                            'speedup': 2000,
                            'action_type': 'Low_Level_Continuous',                        
                            'full_observation': 1,
                            'actions_2d': 0
                            },
                        'SimConfig' : {                        
                            'num_allies': 1,
                            'num_enemies': 1,
                            'enemies_behavior': 'wez_eval_target_max',
                            'agents_behavior' : 'wez_eval_shooter',
                            'agents_config' : { 
                                'blue_agents': 
                                    {                        
                                        'init_position': {'x': 0.0, 'y': 25000.0,'z': 30.0},
                                        'offset_pos': {	'x': 0.0, 'y': 0.0, 'z': 0.0},
                                        'init_hdg': 0.0,                        
                                        'target_position': {'x': 0.0,'y': 25000.0,'z': -30.0},                                        
                                        'rnd_offset_range':{'x': 0.0,'y': 0.0,'z': 0.0},
                                        'rnd_shot_dist_var': 0.0 																						
                                    },                            
                                'red_agents':
                                    { 
                                        'init_position': {'x': 0.0,'y': 25000.0,'z': -30.0},
                                        'offset_pos': {'x': 0.0,'y': 0.0,'z': 0.0},
                                        'init_hdg' : 180.0,                        
                                        'target_position': {'x': 0.0,'y': 25000.0,'z': 30.0},                                        
                                        'rnd_offset_range':{'x': 0.0,'y': 0.0,'z': 0.0},
                                        'rnd_shot_dist_var': 0.0					
                                    }   
                            } 
                            
                        }
                }

                    


In [6]:
env = None
blue_alt_range     = [1000, 50000]
red_alt_range      = [1000, 50000]
angle_off_range    = [-180,  180 ]
aspect_angle_range = [-90, 90]

random.seed(110)
conditions = []
for i in range(10000):
    conditions.append(generate_random_cases(blue_alt_range,red_alt_range, angle_off_range, aspect_angle_range ))
    
final_results = []

for i,cond in enumerate(conditions): 
    
    cases =  generate_cases(cond['blue_alt'], cond['red_alt'], cond['angle_off'], cond['aspect_angle'])

    experimentConfig = { 'runs_per_case': 1, 'cases' : cases }
    
    # Create the GodotExperimentWrapper
    if env == None:
        config_dict['ExperimentConfig'] = experimentConfig
        env = GodotExperimentWrapper(config_dict)
    else:
        env.send_sim_config(experimentConfig)

    # Run the experiment
    results = env.watch_experiment()

    # Save the experiment results to a file
    #with open('experiment_results.json' , "w") as file:
    #    json.dump(results, file, indent=4)

    # Flatten the list of lists into a single list of dictionaries
    flat_data = [item for sublist in results for item in sublist]
    # Create the DataFrame
    df = pd.DataFrame(flat_data)

    item_index = df.loc[df['reds_killed'] == 1].index.max() 
    
    Rmax = cases[item_index]['agents_config']['red_agents']['init_position']['z'] - 30.0
            
    cond["Rmax"] = Rmax
    final_results.append(cond)
    if (i%10) == 0:
        print(i, end=".")
    else:
        print(".", end="")
    #env.close()
    #env = None

#env.close()
print(final_results)

waiting for remote GODOT connection on port 11573
connection established
0..........10..

In [ ]:
df =  pd.DataFrame(final_results)
print(df)
df.to_csv('Rmax_2380_v1.csv')

          blue_alt       red_alt   angle_off  aspect_angle  Rmax
0      8137.793500  23291.423221   97.482170     36.992381 -28.5
1     36865.989679  22242.207310  108.007365      5.922255 -29.5
2      4927.531750  23341.348178 -162.760494     77.933232 -18.0
3     47406.822294  17432.188021  -68.613866     48.243267 -19.5
4     10989.606910   9744.577385 -112.105831    -27.539198 -33.0
...            ...           ...         ...           ...   ...
2379  14513.529301  10492.202596  -71.696945    -56.116514 -30.5
2380   9488.327484   7138.799855  -71.479225     -5.537869 -19.0
2381  24391.237336  47060.104046  -27.316811     76.584536 -16.5
2382  16480.143130  28208.153704  144.087660     79.555532 -29.5
2383   3026.809851   8146.190123  153.899245    -77.026650 -17.5

[2384 rows x 5 columns]
